In [ ]:
import time
import numpy as np
import cv2
import torch
import torch.backends.cudnn as cudnn
from models.experimental import attempt_load
from utils.general import non_max_suppression
import pandas as pd

In [ ]:
camera = 0 # webcam
weights = 'best_face.pt'
device = torch.device('cpu')

model = attempt_load(weights, map_location=device)  # load FP32 model
stride = int(model.stride.max())  # model stride
cudnn.benchmark = True

# Capture with opencv and detect object
cap = cv2.VideoCapture(camera)
width, height = (320, 240) # quality 
cap.set(3, width) # width
cap.set(4, height) # height

In [ ]:
#No skype / No telegram
while(cap.isOpened()):
    ret, frame = cap.read()
    
    if ret ==True:
        now = time.time()
        img = torch.from_numpy(frame).float().to(device).permute(2, 0, 1)
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        pred = model(img, augment=False)[0]
        pred = non_max_suppression(pred, 0.39, 0.45, classes=0, agnostic=True) # img, conf, iou, classes, ...

        df = pd.DataFrame(pred[0].numpy())
        print('time -> ', time.time()-now)

        if df.size > 0:
            time_stamp = int(time.time())
            fcm_photo = f'detected/{time_stamp}.png'
            cv2.imwrite(fcm_photo, frame)
            print(df)

        time.sleep(2)
    else:
        break
    
cap.release()

In [ ]:
# Logging into skype
from skpy import Skype
from getpass import getpass

sk = Skype()
sk.conn.liveLogin("glados@procrastinator.me", getpass())
#sk.conn.soapLogin("cibarcena4@outlook.com", getpass())
#sk.getSkypeToken()

carlos = sk.contacts["live:cibarcena4_1"]

print(sk.conn)

In [ ]:
# Skype
while(cap.isOpened()):
    ret, frame = cap.read()
    
    if ret ==True:
        now = time.time()
        img = torch.from_numpy(frame).float().to(device).permute(2, 0, 1)
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        pred = model(img, augment=False)[0]
        pred = non_max_suppression(pred, 0.39, 0.45, classes=0, agnostic=True) # img, conf, iou, classes, ...

        df = pd.DataFrame(pred[0].numpy())
        print('time -> ', time.time()-now)

        if df.size > 0:
            time_stamp = int(time.time())
            fcm_photo = f'detected/{time_stamp}.png'
            print(fcm_photo)
            cv2.imwrite(fcm_photo, frame)
            carlos.chat.sendMsg('Someone detected!')
            with open(fcm_photo, "rb") as f:
                carlos.sendFile(f, time_stamp, image=True)
            print(df)

        time.sleep(4)
    else:
        break
    
cap.release()